In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras import regularizers
from sklearn.metrics import roc_curve, auc
from keras.layers import Activation
from keras import backend as K

In [ ]:
# load saved df
df = pd.read_pickle('test.pkl')
print (df.keys())

In [ ]:
# filter events
ieta=24
df = df.loc[abs(df[b'hcal_ieta']) == ieta]

df = df.loc[df["b'hcal_edepth1'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth1'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth2'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth2'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth3'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth3'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth4'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth4'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth5'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth5'_y"] > 0.00001]

df = df.loc[df["b'hcal_edepth6'_x"] > 0.00001]
df = df.loc[df["b'hcal_edepth6'_y"] > 0.00001]

if ieta in range (26,28):
    df = df.loc[df["b'hcal_edepth7'_x"] > 0.00001]
    df = df.loc[df["b'hcal_edepth7'_y"] > 0.00001]


In [ ]:
df["b'hcal_edepth1'_x"]=np.log(10000*df["b'hcal_edepth1'_x"])
df["b'hcal_edepth1'_y"]=np.log(10000*df["b'hcal_edepth1'_y"])

df["b'hcal_edepth2'_x"]=np.log(10000*df["b'hcal_edepth2'_x"])
df["b'hcal_edepth2'_y"]=np.log(10000*df["b'hcal_edepth2'_y"])

df["b'hcal_edepth3'_x"]=np.log(10000*df["b'hcal_edepth3'_x"])
df["b'hcal_edepth3'_y"]=np.log(10000*df["b'hcal_edepth3'_y"])

df["b'hcal_edepth4'_x"]=np.log(10000*df["b'hcal_edepth4'_x"])
df["b'hcal_edepth4'_y"]=np.log(10000*df["b'hcal_edepth4'_y"])

df["b'hcal_edepth5'_x"]=np.log(10000*df["b'hcal_edepth5'_x"])
df["b'hcal_edepth5'_y"]=np.log(10000*df["b'hcal_edepth5'_y"])

df["b'hcal_edepth6'_x"]=np.log(10000*df["b'hcal_edepth6'_x"])
df["b'hcal_edepth6'_y"]=np.log(10000*df["b'hcal_edepth6'_y"])

if ieta in range (26,28):
    df["b'hcal_edepth7'_x"]=np.log(10000*df["b'hcal_edepth7'_x"])
    df["b'hcal_edepth7'_y"]=np.log(10000*df["b'hcal_edepth7'_y"])

In [ ]:
# keep originals before scaling
df['hcal_edepth1_un'] = df["b'hcal_edepth1'_x"]
df['hcal_edepth2_un'] = df["b'hcal_edepth2'_x"]
df['hcal_edepth3_un'] = df["b'hcal_edepth3'_x"]
df['hcal_edepth4_un'] = df["b'hcal_edepth4'_x"]
df['hcal_edepth5_un'] = df["b'hcal_edepth5'_x"]
df['hcal_edepth6_un'] = df["b'hcal_edepth6'_x"]
df['hcal_edepth7_un'] = df["b'hcal_edepth7'_x"]

df['hcal_ieta_un'] = df[b'hcal_ieta']

In [ ]:
# scaling input vars
cols_to_minmax =[b'pt_of_muon',b'eta_of_muon',b'phi_of_muon',b'energy_of_muon',b'hcal_ieta',
                b'hcal_iphi',     b'IsolationR04',     b'IsolationR03',
                b'ecal_3into3',      b'hcal_3into3', 
                b'ecal_3x3',         b'hcal_1x1',
               "b'hcal_edepth1'_x", "b'hcal_edepth2'_x", "b'hcal_edepth3'_x",
               "b'hcal_edepth4'_x", "b'hcal_edepth5'_x", "b'hcal_edepth6'_x",
               "b'hcal_edepth7'_x"]


df[cols_to_minmax] = df[cols_to_minmax].apply(lambda x: (x - x.min()) /  (x.max() - x.min()))

In [ ]:
#print(ieta)
for i in (df.keys()):
    print (i)
    if ieta in range (22,26):
        if i == "b'hcal_edepth7'_x":
            continue
        if i == "b'hcal_edepth7'_y":
            continue
        if i == "b'hcal_edepth7'_un":
            continue
    plt.hist(df[i],bins=100,label=str(i),alpha=0.4)
    plt.yscale("log")
    plt.legend()
    plt.show()

In [ ]:
data = df.values
print (data)
ntest = 20000
testindx = data.shape[0] - ntest

X_train_depth1 = data[:testindx,np.r_[0:12,12]]
X_train_depth2 = data[:testindx,np.r_[0:12,13]]
X_train_depth3 = data[:testindx,np.r_[0:12,14]]
X_train_depth4 = data[:testindx,np.r_[0:12,15]]
X_train_depth5 = data[:testindx,np.r_[0:12,16]]
X_train_depth6 = data[:testindx,np.r_[0:12,17]]
X_train_depth7 = data[:testindx,np.r_[0:12,18]]

Y_train = data[:testindx,20]               
X_test = data[testindx:,:]
print (X_test.shape) 
#print ("shape of X_train:",X_train.shape)
print ("shape of Y_train:",Y_train.shape)

In [ ]:
# sample weights
histoAR = Y_train.copy()
nbins = 4 ### works best[4 with no sqrt / 500 with sqrt normed] / 1000 with norm*100 / 4000 ok no sqrt
true_hist = np.histogram(histoAR,bins = nbins,range=(np.min(Y_train)-0.001,np.max(Y_train)+0.001))
binweight = true_hist[0][np.digitize(histoAR,true_hist[1]) - 1]
print ("true_hist[0] = ", true_hist[0])
binweight = 1/binweight
binweight = 100*binweight/np.sum(np.unique(binweight)) ## 5000/5 ok
print("sum weight:",np.sum(np.unique(binweight)))
print("true_hist",true_hist[0])
print("binweight",binweight)
print("weight_vals:",np.unique(binweight))
plt.hist(histoAR,bins=nbins)
for i in true_hist[1]:
    plt.axvline(i,color='r')
plt.yscale("log")

In [ ]:
true_PU_depth1=df['hcal_edepth1_un']
true_PU_depth2=df['hcal_edepth2_un']
true_PU_depth3=df['hcal_edepth3_un']
true_PU_depth4=df['hcal_edepth4_un']
true_PU_depth5=df['hcal_edepth5_un']
true_PU_depth6=df['hcal_edepth6_un']
true_PU_depth7=df['hcal_edepth7_un']

depth1 = 'Depth: 1'
depth2 = 'Depth: 2'
depth3 = 'Depth: 3'
depth4 = 'Depth: 4'
depth5 = 'Depth: 5'
depth6 = 'Depth: 6'
depth7 = 'Depth: 7'


In [ ]:
import keras.backend as kb
def custom_loss_depth1(preds_depth1,targets):
    return (kb.square(preds_depth1-targets))

def custom_loss_depth2(preds_depth2,targets):
    return (kb.square(preds_depth2-targets))

def custom_loss_depth3(preds_depth3,targets):
    return (kb.square(preds_depth3-targets))

def custom_loss_depth4(preds_depth4,targets):
    return (kb.square(preds_depth4-targets))

def custom_loss_depth5(preds_depth5,targets):
    return (kb.square(preds_depth5-targets))

def custom_loss_depth6(preds_depth6,targets):
    return (kb.square(preds_depth6-targets))

def custom_loss_depth7(preds_depth7,targets):
    return (kb.square(preds_depth7-targets))

In [ ]:
def My_model(X_train):
    PYTHONHASHSEED=0
    from numpy.random import seed
    seed(1)
    import tensorflow as tf
    tf.random.set_seed(2)

    from keras.layers import LeakyReLU
    from keras import optimizers
    
    model = Sequential()
    model.add(Dense(40, input_shape=(X_train.shape[1],)))
    model.add(LeakyReLU(alpha=0.2))
    #model.add(Dropout(0.1))

    model.add(Dense(600))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.1))

    model.add(Dense(600))
    model.add(LeakyReLU(alpha=0.2))
    #model.add(Dropout(0.1))

    model.add(Dense(20))
    model.add(LeakyReLU(alpha=0.2))
    #model.add(Dropout(0.1))

    model.add(Dense(1))
    
    return model

In [ ]:
def model_loss(history_depth):
    print(history_depth.history.keys())
    # summarize history for accuracy
    plt.plot(history_depth.history['loss'])
    plt.plot(history_depth.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt1 = plt.show()
    return plt1

In [ ]:
def distribution(true_PU, preds, targets, title_depth):
    from matplotlib import gridspec
    %matplotlib inline
    i=0

    fig = plt.figure(figsize=(6,6))
    gs = gridspec.GridSpec(2,1, height_ratios=[2, 1]) 
    ax = plt.subplot(gs[0])
    
    a=round(np.mean(targets[:,i]),3)
    ax.hist(targets[:,i], range=(0,20), bins=100, color = 'red', label='truth no PU, Mean='+str(a),histtype='step',linewidth=1.5)
    
    b=round(np.mean(preds[:,i]),3)
    ax.hist(preds[:,i], range=(0,20), bins=100, color = 'green', label='predicted, Mean='+str(b),histtype='step',linewidth=1.5)
    
    plt.hist(true_PU, range=(0,20), bins=100, color = 'yellow', label='truth PU',histtype='step',linewidth=1.5)
    
    ax.legend(loc='upper right')
    ax.set_yscale('log')    
    
    plt.title('ieta: '+str(ieta), loc='left')
    plt.title(str(title_depth), loc='right')
    ax1 = plt.subplot(gs[1])
    
    rat = (preds[:,i]/targets[:,i])
    ax1.plot(rat, 'o')
    ax1.axis([0, 20, -1, 2])
    plt.ylabel('predicted / truth no PU')
    ax1.plot([0, 20], [1, 1])
    plt.savefig('Plots/Final/ieta'+str(ieta)+'_'+str(title_depth)+'.png')
    plt.show()
    
    return plt
    

In [ ]:
def model_save(model_name,title_depth):
    import os
    if not os.path.exists('models'):
        os.makedirs('models')
    model_name.save('models/model_ieta'+str(ieta)+'_'+str(title_depth)+'.h5')

In [ ]:
model1=My_model(X_train_depth1)
model2=My_model(X_train_depth2)
model3=My_model(X_train_depth3)
model4=My_model(X_train_depth4)
model5=My_model(X_train_depth5)
model6=My_model(X_train_depth6)
model7=My_model(X_train_depth7)

In [ ]:
model1.compile(loss=custom_loss_depth1,optimizer='adam')
model1.summary()
print ("fitting now=========>")
history1 = model1.fit(X_train_depth1,Y_train , batch_size=5000, epochs=500, validation_split=0.2,
                      verbose=1,sample_weight=binweight)



model2.compile(loss=custom_loss_depth2,optimizer='adam')
model2.summary()
print ("fitting now=========>")
history2 = model2.fit(X_train_depth2,Y_train , batch_size=5000, epochs=500, validation_split=0.2,
                      verbose=1,sample_weight=binweight)



model3.compile(loss=custom_loss_depth3,optimizer='adam')
model3.summary()
print ("fitting now=========>")
history3 = model3.fit(X_train_depth3,Y_train , batch_size=5000, epochs=500, validation_split=0.2,
                      verbose=1,sample_weight=binweight)



model4.compile(loss=custom_loss_depth4,optimizer='adam')
model4.summary()
print ("fitting now=========>")
history4 = model4.fit(X_train_depth4,Y_train , batch_size=5000, epochs=500, validation_split=0.2,
                      verbose=1,sample_weight=binweight)


model5.compile(loss=custom_loss_depth5,optimizer='adam')
model5.summary()
print ("fitting now=========>")
history5 = model5.fit(X_train_depth5,Y_train , batch_size=5000, epochs=500, validation_split=0.2,
                      verbose=1,sample_weight=binweight)



model6.compile(loss=custom_loss_depth6,optimizer='adam')
model6.summary()
print ("fitting now=========>")
history6 = model6.fit(X_train_depth6,Y_train , batch_size=5000, epochs=500, validation_split=0.2,
                      verbose=1,sample_weight=binweight)


model7.compile(loss=custom_loss_depth7,optimizer='adam')
model7.summary()
print ("fitting now=========>")
history7 = model7.fit(X_train_depth7,Y_train , batch_size=5000, epochs=500, validation_split=0.2,
                      verbose=1,sample_weight=binweight)



In [ ]:
model_loss(history1)
model_loss(history2)
model_loss(history3)
model_loss(history4)
model_loss(history5)
model_loss(history6)
if ieta in range (26,28):
    model_loss(history7)

In [ ]:
# evaluate model
preds1 = model1.predict(X_test[:,np.r_[0:12,12]])
preds2 = model2.predict(X_test[:,np.r_[0:12,13]])
preds3 = model3.predict(X_test[:,np.r_[0:12,14]])
preds4 = model4.predict(X_test[:,np.r_[0:12,15]])
preds5 = model5.predict(X_test[:,np.r_[0:12,16]])
preds6 = model6.predict(X_test[:,np.r_[0:12,17]])
preds7 = model7.predict(X_test[:,np.r_[0:12,18]])

targets = X_test[:,20]
uncorrected = X_test[:,27]
targets = targets.reshape(targets.shape[0],1)
uncorrected = uncorrected.reshape(uncorrected.shape[0],1)

In [ ]:
distribution(true_PU_depth1, preds1, targets, depth1)
distribution(true_PU_depth2, preds2, targets, depth2)
distribution(true_PU_depth3, preds3, targets, depth3)
distribution(true_PU_depth4, preds4, targets, depth4)
distribution(true_PU_depth5, preds5, targets, depth5)
distribution(true_PU_depth6, preds6, targets, depth6)
if ieta in range (26,28):
    distribution(true_PU_depth7, preds7, targets, depth7)


In [ ]:
model_save(model1, depth1)
model_save(model2, depth2)
model_save(model3, depth3)
model_save(model4, depth4)
model_save(model5, depth5)
model_save(model6, depth6)
if ieta in range (26,28):
    model_save(model7, depth7)